## Manipulação de Dados Ouro
Para exemplficar melhor o uso do Pandas vamos fazer a manipulação de dados com os dados de Ouro.

Inicialmente importamos os arquivos como de costume

In [68]:
import pandas as pd

file_gold_production = 'files/gold-production.csv'
file_gold_price     =  'files/Gold_price.csv'

df_gold  = pd.read_csv(file_gold_production)
df_price = pd.read_csv(file_gold_price)

df_gold



,Entity,Code,Year,Gold Production (Clio-Infra & USGS)
0,Afghanistan,AFG,2013,0
1,Afghanistan,AFG,2014,0
2,Afghanistan,AFG,2015,0
3,Algeria,DZA,1681,0
4,Algeria,DZA,1682,0
...,...,...,...,...
36473,Zimbabwe,ZWE,2011,13
36474,Zimbabwe,ZWE,2012,8
36475,Zimbabwe,ZWE,2013,14
36476,Zimbabwe,ZWE,2014,15


#### Vou renomear a coluna da produção de ouro

In [69]:
df_gold.rename(columns = {'Gold Production (Clio-Infra & USGS)':'Gold Production'}, inplace=True)
df_gold



,Entity,Code,Year,Gold Production
0,Afghanistan,AFG,2013,0
1,Afghanistan,AFG,2014,0
2,Afghanistan,AFG,2015,0
3,Algeria,DZA,1681,0
4,Algeria,DZA,1682,0
...,...,...,...,...
36473,Zimbabwe,ZWE,2011,13
36474,Zimbabwe,ZWE,2012,8
36475,Zimbabwe,ZWE,2013,14
36476,Zimbabwe,ZWE,2014,15


### **DataFrame do preço**
Como vocês poderão reparar, as Datas não estão no formato devido

In [70]:
df_price['Date'] = pd.to_datetime(df_price['Date'])

### Vamos filtrar os dados para um periodo de data específico
 No caso precisaremos de um `start_date` e `end_date`

In [71]:
start_date = 1970
end_date   = 2010
n_top = 11 ## 10 maiores produtores nesse periodo

df_gold_filtered = df_gold[(df_gold['Year']>=start_date) & (df_gold['Year']<=end_date)]
df_gold_filtered

,Entity,Code,Year,Gold Production
292,Algeria,DZA,1970,0
293,Algeria,DZA,1971,0
294,Algeria,DZA,1972,0
295,Algeria,DZA,1973,0
296,Algeria,DZA,1974,0
...,...,...,...,...
36468,Zimbabwe,ZWE,2006,11
36469,Zimbabwe,ZWE,2007,7
36470,Zimbabwe,ZWE,2008,4
36471,Zimbabwe,ZWE,2009,5


### Vamos criar um ranking dos maiores 

In [72]:


df_gold_group = df_gold_filtered.groupby('Entity')['Gold Production'].sum().reset_index()
df_gold_group = df_gold_group.sort_values(by='Gold Production', ascending=False).reset_index(drop=True)
df_gold_group[1: n_top + 1]

,Entity,Gold Production
1,South Africa,23334
2,United States,7684
3,Russia,6931
4,Australia,6348
5,China,4941
6,Canada,4474
7,Uzbekistan,3552
8,Peru,2517
9,Indonesia,2039
10,Papua New Guinea,1693


### Vamos Criar uma lista com os top paises e devemos inserir um outro dado que seria 
`Rest of World`

In [73]:
df_ranking_countries = df_gold_group[1: n_top + 1]
total_top_countries = df_ranking_countries['Gold Production'].sum()
rest_world_production = df_gold_group.loc[0,'Gold Production'] - total_top_countries

rest_of_world_df = pd.DataFrame({'Entity': ['Rest of World'], 'Gold Production': [rest_world_production]})
df_ranked = pd.concat([df_ranking_countries, rest_of_world_df], ignore_index=True)

df_ranked

,Entity,Gold Production
0,South Africa,23334
1,United States,7684
2,Russia,6931
3,Australia,6348
4,China,4941
5,Canada,4474
6,Uzbekistan,3552
7,Peru,2517
8,Indonesia,2039
9,Papua New Guinea,1693


#### Histograma, Pizza e Serie Temporal com `Plotly`

In [74]:
import plotly.express as px

## pie Chart
pie_fig = px.pie(df_ranked[1:], values='Gold Production', names='Entity', title=f'Proporção dos Maiores Produtores de Ouro ({start_date}-{end_date})')
pie_fig


In [75]:
# Bar Chart
bar_fig = px.bar(df_ranked[1:], x='Entity', y='Gold Production', title=f'Produção Total dos Maiores Produtores de Ouro ({start_date}-{end_date})')
bar_fig

#### Para o gráfico da série temporal temos que filtrar os dados 

In [76]:
list_ranking = df_ranking_countries['Entity'].tolist()
df_plot = df_gold_filtered[df_gold_filtered['Entity'].isin(list_ranking)]
df_plot

,Entity,Code,Year,Gold Production
1629,Australia,AUS,1970,19
1630,Australia,AUS,1971,21
1631,Australia,AUS,1972,24
1632,Australia,AUS,1973,17
1633,Australia,AUS,1974,16
...,...,...,...,...
34793,Uzbekistan,UZB,2006,77
34794,Uzbekistan,UZB,2007,73
34795,Uzbekistan,UZB,2008,73
34796,Uzbekistan,UZB,2009,73


In [77]:
  # Line Chart
line_fig = px.line(df_plot, x='Year', y='Gold Production', color='Entity', title='Produção de Ouro ao Longo do Tempo')
line_fig


#### matriz de Corr


In [78]:
# Correlation Matrix
df_corr = df_plot.pivot(index='Year', columns='Entity', values='Gold Production').corr().round(2)
correlation_fig = px.imshow(df_corr, text_auto=True, aspect="auto", title='Matriz de Correlação dos Maiores Produtores de Ouro')
# Atualizar o layout para garantir que a figura seja quadrada
correlation_fig.update_layout(
    height=600,  # Altura da figura
    width=600    # Largura da figura
)

correlation_fig


### HeatMap com mapa Mundi

In [79]:

df_map = df_gold_group[df_gold_group["Entity"] != "World"]

# Criação do gráfico de calor com mapa-múndi
fig = px.choropleth(
    df_map,
    locations="Entity",
    locationmode="country names",
    color="Gold Production",
    hover_name="Entity",
    color_continuous_scale="Viridis",
    title="Gold Production by Country"
)

# Atualizar o layout para uma barra de cores horizontal e aumentar o tamanho do mapa
fig.update_layout(
    coloraxis_colorbar=dict(
        orientation='h',  # Orientação horizontal
        xanchor='left',
        x=-0.2,
        yanchor='bottom',
        y=-0.3
    ),
    margin=dict(l=20, r=20, t=50, b=50),
    height=600,  # Aumentar a altura do gráfico
    width=1000   # Aumentar a largura do gráfico
)


# Mostrar o gráfico
fig.show()